In [0]:
# Importações das functions

from pyspark.sql.functions import col, year, quarter, when

In [0]:
from pyspark.sql.functions import sum as sum_, col

In [0]:
# Lendo DataFrame da tabela pedidos na camada silver

pedidos_df = spark.table("ecommerce.bronze.pedidos")

In [0]:
# Tranformações da tabela pedidos

pedidos_df = (
    pedidos_df
    
    # Garantir tipo numérico
    .withColumn("valor_total", col("valor_total").cast("decimal(12,2)"))
    .withColumn("desconto", col("desconto").cast("decimal(12,2)"))
    
    # Valor líquido
    .withColumn(
        "valor_liquido",
        col("valor_total") - col("desconto")
    )
    
    # Ano e trimestre
    .withColumn("ano", year(col("data_pedido")))
    .withColumn("trimestre", quarter(col("data_pedido")))
    
    # Classificação do ticket
    .withColumn(
        "ticket_classificacao",
        when(col("valor_liquido") < 200, "Baixo")
        .when((col("valor_liquido") >= 200) & (col("valor_liquido") <= 1000), "Médio")
        .when(col("valor_liquido") > 1000, "Alto")
        .otherwise("Indefinido")
    )
)

In [0]:
#Salvando DataFrame na camada silver
pedidos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.pedidos")